In [1]:
import errno
import os
import pandas as pd
import pycountry

# Carga de datos del .csv original.
df = pd.read_csv("data.csv")
df.head(3)

# Creación de la carpeta para almacenar los ficheros
# .xlsx con los datos procesados para la visualización.
try:
    os.mkdir('dfs')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [2]:
%%capture --no-display

### TRANSFORMACIÓN DE VARIABLES ###
# "Age"
df["Age"] = df["Age"].str.extract(r'\((.*)\)')
df["Age"] = df["Age"].replace(["Adult, Older Adult","Child, Adult",
                                   "Child, Adult, Older Adult",
                                   "Older Adult"],
                                  ["Adult",
                                   "All ages",
                                   "All ages","Adult"])
# "Country"
def extract_country(string):
    for country in pycountry.countries:
        if country.name in string:
            return country.alpha_2
        
df["Locations"] = df["Locations"].astype("string").fillna('')
df['Country'] = df['Locations'].apply(extract_country)



# "Study Type"
df = df[df['Study Type'].isin(["Interventional","Observational"])]


# "Dates"
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['start_month_year'] = df['Start Date'].dt.strftime('%m/%Y').apply(pd.to_datetime)
df = df[(df['start_month_year'] > "31/12/2019") & (df['start_month_year'] < "01/01/2021")]

In [3]:
## 1. ¿Qué países desarrollaron más estudios relacionados con el COVID-19
## durante el primer año de pandemia?


# Número de estudios por país
c_dict = df.groupby("Country").size().to_dict()
df_c = pd.read_excel("countries_flourish.xlsx")
df_c.columns

df_c["Country name ISO"] = df_c['Country name'].apply(extract_country)
df_c["Studies"] = [c_dict[i] if i in c_dict.keys() else 0 for i in df_c["Country name ISO"]]
df_c = df_c.sort_values(by=["Studies"],ascending=False).drop_duplicates(keep='first',subset=['Country name ISO'])

df_c.to_excel("dfs/Estudios por país.xlsx")
df_c.head(10)

,Country name,Country name ISO,Studies
243,United States of America,US,757
79,France,FR,590
241,United Kingdom,GB,206
111,Italy,IT,200
217,Spain,ES,175
67,Egypt,EG,171
46,China,CN,164
234,Turkey,TR,159
40,Canada,CA,159
31,Brazil,BR,138


In [4]:
## 2. ¿Cuántos estudios se desarrollaron durante los primeros 12
## meses de pandemia?


# Número de estudios a lo largo de los meses
study_type_df = pd.get_dummies(df["Study Type"], prefix='Study Type')
study_type_df["date"] = df["start_month_year"]
study_type_df = study_type_df.groupby('date').sum()
study_type_df["Total"] = study_type_df["Study Type_Interventional"] + study_type_df["Study Type_Observational"]

study_type_df.to_excel("dfs/Evolución del número de estudios.xlsx")
study_type_df

,Study Type_Interventional,Study Type_Observational,Total
date,,,
2020-01-01,28.0,34.0,62.0
2020-02-01,50.0,52.0,102.0
2020-03-01,119.0,300.0,419.0
2020-04-01,436.0,413.0,849.0
2020-05-01,360.0,312.0,672.0
2020-06-01,275.0,258.0,533.0
2020-07-01,243.0,150.0,393.0
2020-08-01,198.0,84.0,282.0
2020-09-01,222.0,107.0,329.0


In [5]:
## 3. ¿Qué tipo de estudios se desarrolló en mayor medida:
## ensayos clínicos o estudios epidemiológicos?

type_df = df['Study Type'].value_counts()
type_df.to_excel("dfs/Estudios por tipo.xlsx")
type_df

Interventional    2473
Observational     1992
Name: Study Type, dtype: int64

In [6]:
## 4. ¿Qué características sociodemográficas tenían las poblaciones de 
## estudio?

# Número de estudios por edad y tipo
age_type_df = pd.crosstab(df['Age'], df['Study Type'])
age_type_df.to_excel("dfs/Estudios por edades.xlsx")
age_type_df

Study Type,Interventional,Observational
Age,,
Adult,2217,1452
All ages,137,181
Child,28,34


In [7]:
## 4. ¿Qué características sociodemográficas tenían las poblaciones de 
## estudio?


# Número de estudios por género y tipo
sex_type_df = pd.crosstab(df['Gender'], df['Study Type'])
sex_type_df.to_excel("dfs/Estudios por sexo.xlsx")
sex_type_df

Study Type,Interventional,Observational
Gender,,
All,2411,1901
Female,38,71
Male,24,12
